In [4]:
from __future__ import division
import numpy as np
import sys

X_train = np.genfromtxt('X_train.csv', delimiter=",")
y_train = np.genfromtxt('y_train.csv')
X_test = np.genfromtxt('X_test_all.csv', delimiter=",")



In [9]:
## can make more functions if required


def pluginClassifier(X_train, y_train, X_test):    
  class_count = 10
  
  n = np.unique(y_train,return_counts=True)[1]# number of samples in each class
  pi = n/ y_train.shape[0] # prior probality of each class
  mask = np.array([[y == class_index for y in y_train] for class_index in range(class_count)]) * 1
  mu = np.zeros((class_count, X_train.shape[1])) #mu is the avg of all feature vector in class k. 
  for class_index in range(class_count):
    mu[class_index] = np.sum(mask[class_index].reshape(-1,1) * X_train, axis=0) / n[class_index]

  sigma = np.zeros((class_count,X_train.shape[1],X_train.shape[1]))
  for class_index in range(class_count):
    for i in range(mask[class_index].shape[0]):
        if mask[class_index][i]>0: # only compute sample i belong to current class so mask_index is 1 otherwise 0
          sigma[class_index] += np.matmul(
            (mu[class_index]-X_train[i]).reshape(-1,1),  
            (mu[class_index]-X_train[i]).reshape(1,-1)
            )# here we do matrix multiplication where we do reshaping the matrix for multiply
    
  sigma[class_index] = sigma[class_index] / n[class_index]
    
    
  sigma_inv = np.array([np.linalg.inv(sigma[i]) for i in range(sigma.shape[0])])
  sigma_inv_sqrt_det = np.sqrt(np.array([np.linalg.det(np.linalg.inv(sigma[i])) for i in range(sigma.shape[0])]))
    
  prob = np.zeros((X_test.shape[0], class_count))
  for i in range(X_test.shape[0]):
        for class_index in range(class_count):
            prob[i][class_index] = (pi[class_index] / sigma_inv_sqrt_det[class_index]) * \
            np.exp(-0.5 * np.linalg.multi_dot([
                (X_test[i] - mu[class_index]).reshape(1, -1),
                sigma_inv[class_index],
                (X_test[i] - mu[class_index]).reshape(-1, 1)
            ])[0][0])




  return prob / np.sum(prob, axis=1).reshape(-1,1)


final_outputs = pluginClassifier(X_train, y_train, X_test) # assuming final_outputs is returned from function



In [10]:
print(final_outputs)

[[9.97202888e-03 8.04436052e-07 6.06021656e-02 ... 6.98405178e-02
  5.40455299e-01 1.47744342e-32]
 [1.03331510e-02 1.89545089e-06 5.85910438e-02 ... 7.49070133e-02
  5.30201589e-01 1.03232759e-27]
 [9.76460145e-03 2.43821143e-06 5.74442517e-02 ... 7.37812935e-02
  5.38718546e-01 8.83798197e-25]
 ...
 [8.89626489e-03 5.81394072e-06 5.63891336e-02 ... 7.68690508e-02
  5.39745421e-01 3.79707392e-15]
 [7.53991242e-03 4.92579756e-06 5.52068793e-02 ... 7.75816593e-02
  5.46480797e-01 5.25591197e-16]
 [8.92661568e-03 5.82961504e-06 5.62887687e-02 ... 7.68599898e-02
  5.39156569e-01 2.58808045e-15]]
